# Times of India Article Display Interface

In [1]:
from toidisplay import display_requested_articles, load_metadata
import pandas as pd

## Define custom functions:

If you have a list or custom script to output a list of articles, a custom input function to run after display of each article (say, a coding assignment), or  

In [2]:
# Add list-defining function here:

def get_display_list():
    
    if 'second_opinion_df' not in globals():
        global second_opinion_df
        second_opinion_df = pd.read_csv('../pogrep/temp/second_opinion.csv', dtype={'second_opinion':'Bool'}, index_col=0)
    
    if 'second_opinion' not in globals():
        global second_opinion
        second_opinion = second_opinion_df.second_opinion.to_dict()

    # read in the training data
    td = pd.read_csv('../pogrep/training_data.csv', dtype='object', index_col='record_id')

    # read in the second_opinion work so far

    request_list = list(set(td.index.tolist()) - set(second_opinion_df.index.tolist()))
    
    return request_list

## Define Input and Save functions (Optional)
If you would like to have a custom input function run after display (say, a coding assignment), and if you would like to save the results of the input function, add them in the cell below. Define them as linked_function() and save_function() respectively. If linked_function remains undefined, the page will default to a "Press enter to continue" dialog box with no save function.

In [3]:
def linked_function(article):
    
    new_judgment = ''
    while type(new_judgment) != bool:
        new_judgment = input("Is this a pogrom narrative? (y or 1 = Yes, n or 0 = No)")
        if new_judgment == 'y' or new_judgment == '1':
            new_judgment = True
        elif new_judgment == 'n' or new_judgment == '0':
            new_judgment = False
        else:
            print("I'm not sure how to interpret that.")
    second_opinion.update({int(article):new_judgment})
    
def save_function():  
    opinion = {'second_opinion': second_opinion}
    global second_opinion_df
    second_opinion_df = pd.DataFrame(opinion)
    second_opinion_df.to_csv('../pogrep/temp/second_opinion.csv')

## Display Articles

In [ ]:
display_requested_articles(display_list=get_display_list, linked_function=linked_function, save_function=save_function)

Loading metadata...
done 

Unpacking requested pdfs...
Done!
Here's article 1 of 15, in set 1 of 31:


Article ID: 614153620 	 Published: 1986-02-15
Object Types:	 feature, article


In [6]:
second_opinion_df.second_opinion.value_counts()

False    1239
True      141
Name: second_opinion, dtype: int64